In [8]:
import pandas as pd
import os
from thoughtfulness_features import *

# Read all the thoughtful comments labelled files

In [15]:
path_parent = os.path.dirname(os.path.dirname(os.getcwd()))
file_path = f'{path_parent}/Data/Thoughtful Comments'
files = [file for file in os.listdir(file_path) if file != '.DS_Store' and file.endswith('.xls')]

In [16]:
files

['vaccination.xls',
 'safe_distancing.xls',
 'circuit_breaker.xls',
 'Tracetogether.xls']

# Create thoughtful comment features for every file

In [18]:
test = pd.read_excel(f'{file_path}/circuit_breaker.xls')
test.head()

,Unnamed: 0,comment,Thoughtful?
0,0,🇸🇬New Parc Clematis National Day Promo startin...,0
1,1,"Oh my fucking God, this announcement could not...",0
2,2,Dang it. I was super happy for a while.\n\nIt'...,0
3,3,After a beautiful front 9 we got hit with the ...,0
4,4,"these people must be tagged, if not cannot ent...",0


In [ ]:
for file in files:
    df = pd.read_excel(f'{file_path}/{file}')

    # Creating feature 1 (Structural feature): comment length
    comment_length = []
    for row in df['comment']:
        length = get_comment_length(row)
        comment_length.append(length)
    df['Length'] = comment_length
    
    # Removing rows with zero comment length
    df.drop(df[df.Length == 0].index, inplace=True)

    # Preparation for feature 2 (Lexical feature): comment likelihood
    comment_likelihood = []
    news_unigram = news_articles_unigram(f'{path_parent}/Data')
    # Preparation for feature 3 (Syntatic feature): number of verbs
    for row 